In [15]:
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris 
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 
import pandas as pd
import numpy as np

In [2]:
# Load the TF-IDF data from CSV files
X_train = pd.read_csv('train_tfidf.csv')
X_test = pd.read_csv('test_tfidf.csv')

X_train.drop(X_train.columns[0], axis=1, inplace=True)
X_test.drop(X_test.columns[0], axis=1, inplace=True)

# Loading the labels
pd_train = pd.read_csv('train_labels.csv').values.ravel()
pd_test = pd.read_csv('test_labels.csv').values.ravel()

pd_train = list(pd_train)
pd_test = list(pd_test)

y_train = [1 if x == "non-suicide" else 0 for x in pd_train]
y_test = [1 if x == "non-suicide" else 0 for x in pd_test]
y_train = np.array(y_train)
y_test = np.array(y_test)

In [3]:
estimator = [] 
#estimator.append(('LR', LogisticRegression(solver ='lbfgs', multi_class ='multinomial', max_iter = 200))) 
estimator.append(('SVC', SVC(probability = True))) 
#estimator.append(('DTC', DecisionTreeClassifier())) 

In [16]:
lr_best = LogisticRegression(max_iter=1000, C = 1, penalty = 'l2', solver = 'newton-cg')
dtclf_best = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf=1, min_samples_split=5, splitter='best')
rf = RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_leaf=2, min_samples_split=10,
                                    n_estimators =  200)
svc = SVC()
best_estimator_bnb = BernoulliNB(alpha=1, binarize=None)

In [5]:
lr_best.fit(X_train, y_train)
accuracy_score(y_test, lr_best.predict(X_test))

0.9150666666666667

In [6]:
dtclf_best.fit(X_train, y_train)
accuracy_score(y_test, dtclf_best.predict(X_test))

0.8536

In [7]:
rf.fit(X_train, y_train)
accuracy_score(y_test, rf.predict(X_test))

0.8582666666666666

In [8]:
accuracy_score(y_train, rf.predict(X_train))

0.8713777777777778

In [17]:
best_estimator_bnb.fit(X_train, y_train)
accuracy_score(y_test, best_estimator_bnb.predict(X_test))

0.8858666666666667

In [20]:
from sklearn.ensemble import StackingClassifier

# Create a list of classifiers
level0 = [('rf',rf), ('best_estimator_bnb', best_estimator_bnb)]
level1 = lr_best

# Create a stacking classifier object
stack_clf = StackingClassifier(estimators=level0, final_estimator=level1, cv=5, n_jobs=-1, passthrough=True)

# Train the classifier
stack_clf.fit(X_train, y_train)

# Predict the validation set
y_pred = stack_clf.predict(X_test)

# Calculate the accuracy
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Test Accuracy: 0.9137333333333333


In [25]:
# Create a voting classifier object
from sklearn.ensemble import VotingClassifier

# Create a list of classifiers
estimators = [('lr_best', lr_best), ('stack_clf',stack_clf), ('rf', rf)]

# Create a voting classifier object
ensemble = VotingClassifier(estimators, voting='hard')

# Train the classifier
ensemble.fit(X_train, y_train)

# Predict the validation set
y_pred = ensemble.predict(X_test)

# Calculate the accuracy
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Test Accuracy: 0.9142666666666667
